# From Data to Paper: Everything you need to go from random data to a LaTeX worthy table and figure

In this python lesson we'll be working within the confines of this Jupyter Notebook to download data from the NASA Exoplanet Archive, plot it in various ways and reproduce the results of this research note: http://iopscience.iop.org/article/10.3847/2515-5172/aac728

If at any point you have questions or would like to see how to do something in particular please speak up or feel free to email me: laura.mayorga@cfa.harvard.edu

## 1. We're going to need some stuff

In [1]:
# This will grant us access to many array handling functions 
# so that we can do math and is almost always loaded
import numpy as np

# We wanna be able to plot stuff!
# We've chosen to rename the module to save us time and make the code clear
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# We need to be able to read and write stuff!
from astropy.table import QTable
from astropy.io import ascii
import pandas as pd

# We'll be doing some computations and these constants are super useful
from astropy.constants import sigma_sb, R_sun, R_earth, R_jup, \
                        L_sun, au, M_sun, M_jup, M_earth, G
    
# For a non-astro-centric unit conversion library, see https://pypi.org/project/unyt
# There's also pint
from astropy import units as u

# Download files here: https://tinyurl.com/laura-python

## Did you know you can call command line stuff from within python?

In [2]:
# These are the magic queries to get you the files you need!
import subprocess as subp

# This will download planets.csv
command = ['wget', 'https://exoplanetarchive.ipac.caltech.edu/cgi-bin/'+\
            'nstedAPI/nph-nstedAPI?table=exoplanets'+\
            '&select=pl_hostname,pl_rade,pl_eqt'+\
            '&where=pl_rade>0 and pl_eqt>0'+\
            '&format=csv', '-O', 'planets.csv']
neastat = subp.call(command)

# This will download sullivan.txt
command = ['wget', 'http://iopscience.iop.org/0004-637X/809/1/77/suppdata/apj516470t6_mrt.txt',
          '-O', 'sullivan.txt']
sullystat = subp.call(command)

# This will download inflation.csv
command = ['wget', 'https://www.dropbox.com/s/i0usuh5xtwk5pfg/jupMax.txt?dl=0',
           '-O', 'inflation.csv']
infstat=subp.call(command)

# This checks if the commands worked!
if neastat!=0 or sullystat!= 0 or infstat != 0:
    print('SOMETHING BROKE ASK FOR HELP!!')
else:
    print('Queries worked successfully')

Queries worked successfully


## 2. Let's get some data!
There are multiple ways of doing this so lets try a couple and see why some might be better than others in this case.

In [3]:
with open('sullivan.txt', 'r') as f:
    sullivan = ascii.read(f.readlines())
with open('planets.csv', 'r') as f:
    nexsci = ascii.read(f.readlines())

inflation = ascii.read('inflation.csv', names=['T', 'eng', 'R'])
    
# Now you should open the files and look at them yourself.
# ascii.read handles the header of sullivan.txt, and is astropy-centric.
# This format is known as Enhanced Character Separated Values
# It handles units and descriptions in data table.

# planets.csv is in the more common Common Separated Value format
# It allows only a simple label header, along with data values

print(sullivan.info)
print(nexsci.info)

<Table length=1984>
  name   dtype   unit                  description                 
------- ------- ------ --------------------------------------------
  RAdeg float64    deg   Right Ascension in decimal degrees (J2000)
  DEdeg float64    deg       Declination in decimal degrees (J2000)
     Rp float64   Rgeo              Planetary radius in Earth units
    Per float64      d                                       Period
      S float64                 Planetary insolation in Earth units
      K float64  m / s                Radial-velocity semiamplitude
     R* float64 solRad                               Stellar radius
   Teff float64      K                Stellar effective temperature
   Vmag float64    mag                    Apparent V band magnitude
   Imag float64    mag                 Apparent I_C_ band magnitude
   Jmag float64    mag                    Apparent J band magnitude
  Ksmag float64    mag                 Apparent K_s_ band magnitude
     DM float64    mag      

In this case both are Tables (analogous to the Pandas DataFrame) and the Sullivan table has all these nice units and information attached. Can we make both of them have units?

Let's try another way

In [4]:
sullivan = QTable.read('sullivan.txt', format='ascii')

nexsci = QTable.read('planets.csv', names=['name', 'Rp', 'Teq'])

# We can change the names and set the units this way!
nexsci['Rp'].unit=u.R_earth
nexsci['Teq'].unit=u.K

print(sullivan.info)
print(nexsci.info)

<QTable length=1984>
  name   dtype   unit                  description                   class  
------- ------- ------ -------------------------------------------- --------
  RAdeg float64    deg   Right Ascension in decimal degrees (J2000) Quantity
  DEdeg float64    deg       Declination in decimal degrees (J2000) Quantity
     Rp float64   Rgeo              Planetary radius in Earth units Quantity
    Per float64      d                                       Period Quantity
      S float64                 Planetary insolation in Earth units   Column
      K float64  m / s                Radial-velocity semiamplitude Quantity
     R* float64 solRad                               Stellar radius Quantity
   Teff float64      K                Stellar effective temperature Quantity
   Vmag float64    mag                    Apparent V band magnitude Quantity
   Imag float64    mag                 Apparent I_C_ band magnitude Quantity
   Jmag float64    mag                    Apparent J ba

That's kind of nice, isn't it? Now instead of each column being a `Column` all the ones with units are `Quantity` objects which have the attached unit information!

Pandas is another nice way of doing things and interfaces well with a lot of other python modules and packages (that I'm sure will come up in a few weeks), but it doesn't keep the units and in this case we would lose a lot of information.

`In []: pd.read_csv('planets.csv', delimiter=',')`

We want to know about the planet radii as a function of equilibrium temperature. What variables can we use to determine that?

Equilibrium temperature, $T_{eq}$ is usually computed as follows:
\begin{equation}
    T_{eq} = T_{eff} (1-A_B)^{1/4} \sqrt{\frac{R_*}{2a}},
\end{equation}
where $T_{eff}$ is the effective temperature of the host star, $A_B$ is the Bond albedo of the planet, $R_*$ is the radius of the host star, and $a$ is orbital semi-major axis.

We have $T_{eff}$ and $R_*$, we can assume an albedo ($A_B = 0$) but we need the semi-major axis. We can get the semi-major axis from understanding the inputs into some of the other provided variables, such as from the `planetary insolation in Earth units`, $S$.

Solar insolation is computed thusly:
\begin{align}
S_{\oplus} &= \frac{L_{\odot}}{4 \pi a_{\oplus}^2} \\
S_{\oplus} &= \frac{4 \pi \sigma  R_{\odot}^2 T_{\odot}^4}{4 \pi a_{\oplus}^2} \\
S_{\oplus} &= \frac{\sigma R_{\odot}^2 T_{\odot}^4}{a_{\oplus}^2}
\end{align}

Thus the semi-major axis can be found by inverting the above equation to:
\begin{align}
a &= \sqrt{\frac{\sigma R_*^2 T_{\rm eff}^4}{S}} \\
a &= R_* T_{\rm eff}^2 \sqrt{\frac{\sigma}{S}}
\end{align}

This assumes that the planet is in a circular orbit.

In [5]:
# Your solution here
#
#T_eq = 

## 3. Let's plot some data!

https://matplotlib.org/2.1.1/gallery/index.html

The first thing you'll discover when you fail to make plot correctly, because it threw an `exception` somewhere or you forgot to `plt.show()`, is that when you fix it and run it, it'll give you a number of plots including all the ones that failed and the one that succeeded. To avoid that pitfall you should always use `plt.close`.
```
plt.close() #to close prior show
plt.plot()
plt.show()  #to show what you plotted
```

For more complicated plots where you'll be wanting to add axes we need the actual object reference.

`fig = plt.figure()`

But this doesn't include the axes object right away, so the simplest and to get you started is:
```
plt.close()
fig, ax = plt.subplots(figsize=(6,5))
plt.show()
```